In [5]:
# Imports and data load
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from IPython.display import HTML

impact_df = pd.read_csv("generateImpactAnalysisData_data (1).csv")
impact_df.head()

,Pain Points,Revenue impact,Margin impact,Cash impact,Customer impact,Strategic impact,Total weighted score,Action Priority
0,Low capture of aftermarket demand,5,3,1,5,2,3.40,Medium
1,Fragmented data and weak commercial focus on s...,2,2,1,5,3,2.40,Low
2,Poor spare-parts availability and long lead times,3,3,5,3,2,3.20,Medium
3,Misaligned or weak customer value proposition,1,1,3,4,1,1.75,Low
4,Organizational design that sidelines service,4,5,2,4,4,4.05,High


Please click on the 'Choose files' button below to upload the `generateImpactAnalysisData_data.csv` file.

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Saving generateImpactAnalysisData_data.csv to generateImpactAnalysisData_data (1).csv
User uploaded file "generateImpactAnalysisData_data (1).csv" with length 4394 bytes


In [6]:
# Prepare data for horizontal Pareto with cumulative percentages
impact_cols_all = [
    "Revenue impact",
    "Margin impact",
    "Cash impact",
    "Customer impact",
    "Strategic impact"
]

traces_by_metric_all_swapped = {}

for col in impact_cols_all:
    agg = impact_df.groupby("Pain Points", as_index=False)[col].mean()
    agg = agg.sort_values(col, ascending=False)

    values = agg[col].values
    cum_values = np.cumsum(values)
    if cum_values[-1] != 0:
        perc = cum_values / cum_values[-1] * 100.0
    else:
        perc = np.zeros_like(cum_values)

    traces_by_metric_all_swapped[col] = {
        "y": agg["Pain Points"].tolist(),
        "x_vals": agg[col].tolist(),
        "x_cum": perc.tolist()
    }

In [7]:
# Build the interactive horizontal Pareto with dropdown and 80% marker
default_col = "Revenue impact"
fig = go.Figure()

for col in impact_cols_all:
    data = traces_by_metric_all_swapped[col]
    visible = True if col == default_col else False

    # Bars (impact)
    fig.add_trace(go.Bar(
        x=data["x_vals"],
        y=data["y"],
        orientation="h",
        name=col,
        marker_color="steelblue",
        visible=visible,
        xaxis="x1"
    ))

    # Cumulative % line
    fig.add_trace(go.Scatter(
        x=data["x_cum"],
        y=data["y"],
        name="Cumulative % of " + col,
        mode="lines+markers",
        marker=dict(color="firebrick"),
        visible=visible,
        xaxis="x2"
    ))

    # 80% vertical cutoff line (on cumulative axis)
    fig.add_trace(go.Scatter(
        x=[80.0, 80.0],
        y=[data["y"][0], data["y"][-1]],
        mode="lines",
        line=dict(color="green", dash="dash"),
        name="80% cutoff for " + col,
        visible=visible,
        xaxis="x2",
        showlegend=False
    ))

buttons = []
for i, col in enumerate(impact_cols_all):
    visibility = []
    for j in range(len(impact_cols_all)):
        # 3 traces per metric: bar, cumulative, 80% line
        visibility.extend([j == i, j == i, j == i])

    buttons.append(dict(
        label=col,
        method="update",
        args=[
            {"visible": visibility},
            {
                "xaxis": {"title": col + " (score)"},
                "xaxis2": {
                    "title": "Cumulative % of total " + col,
                    "overlaying": "x",
                    "side": "top",
                    "range": [0.0, 110.0]
                },
                "title": "Pareto Chart (Horizontal) – " + col + " by Pain Point"
            }
        ]
    ))

fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        buttons=buttons,
        x=0.0,
        y=1.15,
        xanchor="left",
        yanchor="top"
    )],
    title="Pareto Chart (Horizontal) – " + default_col + " by Pain Point",
    yaxis_title="Pain Points",
    xaxis=dict(title=default_col + " (score)", side="bottom"),
    xaxis2=dict(
        title="Cumulative % of total " + default_col,
        overlaying="x",
        side="top",
        range=[0.0, 110.0]
    ),
    hovermode="y unified",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1.0
    )
)

HTML(fig.to_html(include_plotlyjs="cdn"))

In [8]:
# Save interactive chart to HTML file
html_str = fig.to_html(include_plotlyjs="cdn")
file_name = "pareto_all_impacts_horizontal_dropdown_80pct.html"

with open(file_name, "w") as f:
    f.write(html_str)

print("Saved interactive chart to:", file_name)

Saved interactive chart to: pareto_all_impacts_horizontal_dropdown_80pct.html
